In [ ]:
import os

# カレントディレクトリをmagentaに
os.chdir(r"C:\Users\arkw\GitHub\magenta")
print(os.getcwd())

In [ ]:
import numpy as np

def calculate_c_diatonic(notes):
    """Cのダイアトニック・スケールに該当する音符の割合を計算"""
    c_diatonic_scale = {0, 2, 4, 5, 7, 9, 11}
    diatonic_notes = [(note.pitch % 12) in c_diatonic_scale for note in notes]
    return sum(diatonic_notes) / len(notes) if notes else 0

def calculate_note_density(notes, tempo):
    """ノート密度の計算（音数）"""
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    note_steps = notes[-1].end / second_per_beat * 4 # 16分音符のステップ数に変換
    return len(notes) / note_steps if note_steps else 0

def calculate_average_interval(notes):
    """平均音程間隔の計算"""
    intervals = [abs(notes[i].pitch - notes[i-1].pitch) for i in range(1, len(notes))]
    return sum(intervals) / len(intervals) if intervals else 0

def calculate_average_pitch(notes):
    """平均音高の計算"""
    pitches = [note.pitch for note in notes]
    average_pitch = np.mean(pitches)
    
    return average_pitch

def calculate_syncopation(notes, tempo, syncopation_type):
    """全ての音符に対するシンコペーション音符の割合を計算する
    引数:
        notes: pretty_midi.Noteオブジェクトのリスト
        tempo: テンポ。qpmと同義
        syncopation_type: 8th or 16th
    戻り値:
        シンコペーション音符の割合
    """
    syncopated_count = 0 # シンコペーション音符の数
    prev_note_start = 0 # 一つ前の音符の開始時間
    second_per_beat = 60 / tempo # 一拍あたりの秒数
    if syncopation_type == "8th":
        value_for_evaluation = 0.5
    elif syncopation_type == "16th":
        value_for_evaluation = 0.25

    for note in notes:
        is_position = (note.start / second_per_beat) % (second_per_beat * 2) == value_for_evaluation
        interval = (note.start - prev_note_start) / second_per_beat
        if(is_position and interval > value_for_evaluation):
            syncopated_count += 1
        prev_note_start = note.start
    return (syncopated_count / len(notes)) if notes else 0

def calculate_melody_fullness(notes):
    """ノート密度の計算（音の長さ）"""
    # 音符の長さの総計を計算
    total_note_duration = sum(note.get_duration() for note in notes)
    # メロディ全体の長さを計算（最後の音符の終わりから最初の音符の始まりを引く）
    melody_duration = notes[-1].end - notes[0].start
    # melody_fullnessを計算（音符の長さの総計/メロディ全体の長さ）
    melody_fullness = total_note_duration / melody_duration
    
    return melody_fullness

def calculate_staccato_level(notes):
    """
    スタッカート度合いを計算する
    1に近いほどスタッカート気味、0に近いほどレガート気味
    音符間の間隔が0.5秒より大きい場合は計算から除外する
    """
    durations = []
    intervals = []

    for i in range(len(notes) - 1):
        current_note = notes[i]
        next_note = notes[i + 1]
        
        # 音符間の間隔（インターオンセットインターバル）
        interval = next_note.start - current_note.start
        
        # 間隔が0.5秒以下の場合のみ計算に含める
        if interval <= 0.5:
            # 実際の音符の長さ（デュレーション）
            duration = current_note.get_duration()
            durations.append(duration)
            intervals.append(interval)
    
    if not intervals:  # 有効な間隔がない場合
        return 0  # または適切なデフォルト値
    
    # デュレーションとインターバルの比率を計算
    ratios = np.array(durations) / np.array(intervals)
    
    # スタッカート度合いを計算（1から比率の平均を引く）
    staccato_level = 1 - np.mean(ratios)
    
    return staccato_level

def calculate_legato_level(notes):
     """
     レガート度合いを計算する
     1に近いほどレガート気味、0に近いほどスタッカート気味
     音符間の間隔が0.5秒より大きい場合は計算から除外する
     """
     durations = []
     intervals = []
 
     for i in range(len(notes) - 1):
         current_note = notes[i]
         next_note = notes[i + 1]
         
         # 音符間の間隔（インターオンセットインターバル）
         interval = next_note.start - current_note.start
         
         # 間隔が0.5秒以下の場合のみ計算に含める
         if interval <= 0.5:
             # 実際の音符の長さ（デュレーション）
             duration = current_note.get_duration()
             durations.append(duration)
             intervals.append(interval)
     
     if not intervals:  # 有効な間隔がない場合
         return 0  # または適切なデフォルト値
     
     # デュレーションとインターバルの比率を計算
     ratios = np.array(durations) / np.array(intervals)
     
     # レガート度合いを計算
     legato_level = np.mean(ratios)
     
     return legato_level

In [ ]:
import numpy as np

def analyze_tonality(notes):
    # 長調と短調のスケール度数の重み付け
    major_weights = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
    minor_weights = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
    
    # ノートの出現頻度を計算
    pitch_classes = [note.pitch % 12 for note in notes]
    pitch_hist = np.zeros(12)
    for pc in pitch_classes:
        pitch_hist[pc] += 1
    pitch_hist = pitch_hist / np.sum(pitch_hist)
    
    # 各調の相関を計算
    major_correlation = np.correlate(pitch_hist, major_weights)[0]
    minor_correlation = np.correlate(pitch_hist, minor_weights)[0]
    
    # 正規化
    total = major_correlation + minor_correlation
    major_likelihood = major_correlation / total
    minor_likelihood = minor_correlation / total
    
    return major_likelihood, minor_likelihood

In [ ]:
import pretty_midi
import numpy as np

def analyze_tonality_all_keys(notes):
    # 長調と短調のスケール度数の重み付け
    major_weights = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
    minor_weights = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
    
    # ノートの出現頻度を計算
    pitch_classes = [note.pitch % 12 for note in notes]
    pitch_hist = np.zeros(12)
    for pc in pitch_classes:
        pitch_hist[pc] += 1
    pitch_hist = pitch_hist / np.sum(pitch_hist)
    
    # 全ての調に対して相関を計算
    correlations = []
    for i in range(12):
        major_corr = np.correlate(np.roll(pitch_hist, -i), major_weights)[0]
        minor_corr = np.correlate(np.roll(pitch_hist, -i), minor_weights)[0]
        correlations.append((major_corr, minor_corr))
    
    # 最も高い相関を持つ調を見つける
    best_key = max(range(12), key=lambda i: max(correlations[i]))
    best_major_corr, best_minor_corr = correlations[best_key]
    
    # 正規化して長調らしさと短調らしさを計算
    total = best_major_corr + best_minor_corr
    major_likelihood = best_major_corr / total
    minor_likelihood = best_minor_corr / total
    
    # 調の名前を取得
    key_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    best_key_name = key_names[best_key]
    
    return major_likelihood, minor_likelihood

# 使用例
# midi_data = pretty_midi.PrettyMIDI('path_to_your_midi_file.mid')
# notes = midi_data.instruments[0].notes
# major_likelihood, minor_likelihood, key = analyze_tonality_all_keys(notes)
# print(f"最も可能性の高い調: {key}")
# print(f"長調らしさ: {major_likelihood:.2f}")
# print(f"短調らしさ: {minor_likelihood:.2f}")

In [ ]:
import music21

def detect_key(melody_notes):
    """
    Krumhansl-Schmucklerアルゴリズムを用いたキー検出と相関の出力
    """
    # メロディを音符のリストとして受け取る
    stream = music21.stream.Stream()
    for note in melody_notes:
        # pretty_midiのNoteオブジェクトから音高を取得し、music21のNoteに変換
        pitch = music21.pitch.Pitch(note.pitch)
        stream.append(music21.note.Note(pitch))
    
    # キーを分析
    key = stream.analyze('key')
    key_correlation = key.correlationCoefficient
    return key.tonic.name, key.mode, key_correlation

In [ ]:
import pretty_midi

# 例外オブジェクトを作るためのクラスを定義
# 読み込んだMIDIファイルが条件に合わない場合に
# このクラスによって定義される例外が投げられる
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

def analyze_midi(file_path):
    midi_data = pretty_midi.PrettyMIDI(file_path)
    _, tempi = midi_data.get_tempo_changes()
    tempo = int(tempi[0])
    # time_signature = midi_data.time_signature_changes[0]
    if midi_data.instruments:
      instrument = midi_data.instruments[0]

      # 属性計算
      # c_diatonic = calculate_c_diatonic(instrument.notes) # Cのダイアトニック・スケールに該当する音符の割合
      # note_num = len(instrument.notes) # ノートの数
      note_density = calculate_note_density(instrument.notes, tempo) # ノート密度
      # average_interval = calculate_average_interval(instrument.notes) # 平均音程間隔
      # syncopation_16th = calculate_syncopation(instrument.notes, tempo, "16th") # 16分シンコペーション音符の割合
      # syncopation_8th = calculate_syncopation(instrument.notes, tempo, "8th") # 8分シンコペーションノートの割合
      # major_likelihood, minor_likelihood = analyze_tonality_all_keys(instrument.notes)
      tonic, mode, key_correlation = detect_key(instrument.notes)
      average_pitch = calculate_average_pitch(instrument.notes)
      staccato_level = calculate_staccato_level(instrument.notes)
      

      return (file_path, note_density, tonic, mode, key_correlation, average_pitch, staccato_level)
    else:
       # 空の楽譜の場合はこう
       return (file_path, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

In [ ]:
import glob
import os

# CSV書き出し対象のディレクトリ
midi_path = r"C:\Users\arkw\GitHub\magenta\tmp\music_vae\generated\2bar_8192"
files = glob.glob(os.path.join(midi_path, '*.mid'))

In [ ]:
import csv

def write_to_csv(results, csv_path):
    # ディレクトリが存在しない場合は作成する
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
     # CSVファイルに結果を保存
    with open(csv_path, 'w', newline='') as csvfile:  # 書き込みモード
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(results)  # 結果を1行に書き込む

In [ ]:
import pandas as pd

def write_to_csv(results, csv_path):
    # DataFrameを作成してCSVファイルに結果を保存
    df = pd.DataFrame(
        results, 
        columns=['file_path', 
                 'note_density', 
                 'tonic', 
                 'mode', 
                 'key_correlation', 
                 'average_pitch',
                 'staccato_level'])
    df.to_csv(csv_path, index=False)

In [ ]:
import datetime

results = []
for file in files:
    results.append(analyze_midi(file))

now = datetime.datetime.now()
filename = 'tmp/result_' + now.strftime('%Y%m%d_%H%M%S') + '.csv'
write_to_csv(results, filename)

In [ ]:
# 指定された条件を満たすMIDIファイルを絞り込み、別の場所に保存する（必要に応じて）

import pandas as pd
import shutil
import os

# CSVファイルの読み込み
csv_file_path = 'tmp/result_20240529_150740.csv'
df = pd.read_csv(csv_file_path)

# 条件を満たす行をフィルタリング
filtered_df = df[(df['note_num'] >= 4) & (df['average_interval'] < 8)]

# 保存先ディレクトリの指定
destination_dir = r'tmp\music_vae\generated\adjective01\filtered'
os.makedirs(destination_dir, exist_ok=True)

# 条件を満たすMIDIファイルをコピー
for file_path in filtered_df['file_path']:
    shutil.copy(file_path, destination_dir)

print(f"Filtered MIDI files have been saved to {destination_dir}")

In [ ]:
# 属性ベクトルを出力

import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'staccato_level'

csv_path = "tmp/result_20241021_193319.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# 上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# 下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

# dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [path.replace(".mid", ".npy") for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [path.replace(".mid", ".npy") for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)

In [ ]:
# 調については特殊な過程で算出

import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'mode'

csv_path = "tmp/result_20240906_183929.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# Filter the DataFrame for rows where mode is 'major'
major_df = df[df['mode'] == 'major']
sorted_major_df = major_df.sort_values(by='key_correlation', ascending=False)
top_2048_major = sorted_major_df.head(2048)
# major2048に属するfile_pathの一覧を配列で取得
top_25_percent = top_2048_major['file_path'].tolist()

# Filter the DataFrame for rows where mode is 'minor'
minor_df = df[df['mode'] == 'minor']
sorted_minor_df = minor_df.sort_values(by='key_correlation', ascending=False)
top_2048_minor = sorted_minor_df.head(2048)
# minor2048に属するfile_pathの一覧を配列で取得
bottom_25_percent = top_2048_minor['file_path'].tolist()

# dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [path.replace(".mid", ".npy") for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [path.replace(".mid", ".npy") for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)

In [ ]:
# V/A値を計算する

# Valenceの高いメロディを抽出

import pandas as pd
import numpy as np

# CSVファイルの読み込み
df = pd.read_csv('tmp/result_20241115_203538.csv')

# Valenceスコアの計算
def calculate_valence(row):
    # modeのValence影響度を計算
    mode_score = 1.0 if row['mode'] == 'major' else 0.0
    mode_impact = mode_score * row['key_correlation']
    
    # average_pitchの正規化 (0-1の範囲に)
    pitch_normalized = (row['average_pitch'] - df['average_pitch'].min()) / \
                      (df['average_pitch'].max() - df['average_pitch'].min())
    
    # 両要素を組み合わせた総合Valenceスコア (各要素の重みは0.5ずつ)
    return (mode_impact * 0.8) + (pitch_normalized * 0.2)

# Arouosalスコアの計算
def calculate_arousal(row):
    # note_densityの正規化 (0-1の範囲に)
    density_normalized = (row['note_density'] - df['note_density'].min()) / \
                        (df['note_density'].max() - df['note_density'].min())
    
    # staccato_levelの正規化 (0-1の範囲に)
    staccato_normalized = (row['staccato_level'] - df['staccato_level'].min()) / \
                         (df['staccato_level'].max() - df['staccato_level'].min())
    
    # 両要素を組み合わせた総合Arousalスコア (各要素の重みは0.5ずつ)
    return (density_normalized * 0.5) + (staccato_normalized * 0.5)

# Valenceスコアを計算して新しい列として追加
df['valence_score'] = df.apply(calculate_valence, axis=1)

# Arousalスコアを計算して新しい列として追加
df['arousal_score'] = df.apply(calculate_arousal, axis=1)


In [ ]:
print(df)
df.to_csv("tmp/result_20241119_112200.csv")

In [ ]:
# 平均と標準偏差を出力

import numpy as np

# valence_score or arousal_score
attribute = 'arousal_score'
high_or_low = 'high'

# メロディを抽出 (上位25%など)
if high_or_low == 'high':
    va_melodies = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist() # 上位25%
elif high_or_low == 'low':
    va_melodies = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist() # 下位25%

# file_path25個それぞれの末尾を.npyにして、np.loadする
top_npy_files = [path.replace(".mid", ".npy") for path in va_melodies]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# # 全ての配列を縦方向に結合
# all_arrays = np.vstack(top_npy_arrays)  # shape: (2048, 512)
# # 全体の平均を計算
# mean_value = np.mean(all_arrays)
# # 全体の標準偏差を計算
# std_value = np.std(all_arrays)

# print(f"Mean: {mean_value}")
# print(f"Standard Deviation: {std_value}")

# すべての配列を積み重ねて2次元配列にする
stacked_arrays = np.stack(top_npy_arrays, axis=0)  # shape: (2048, 512)
# 平均を計算 (axis=0で各要素の平均を取る)
mean_array = np.mean(stacked_arrays, axis=0)  # shape: (512,)
# 標準偏差を計算 (axis=0で各要素の標準偏差を取る)
std_array = np.std(stacked_arrays, axis=0)  # shape: (512,)

print(f"mean: {mean_array}")
print(f"std: {std_array}")

# 結果を保存
np.save(f"{attribute}_{high_or_low}_mean_array.npy", mean_array)
np.save(f"{attribute}_{high_or_low}_std_array.npy", std_array)

In [ ]:
import pandas as pd
import numpy as np

# 取り出す属性はこれ！
attribute = 'c_diatonic'

csv_path = "magenta/tmp/result_20240708_145215.csv"
df = pd.read_csv(csv_path)
# filtered_csv_path = "tmp/midi16bar_result100.csv"
# df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df[attribute] >= df[attribute].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df[attribute] <= df[attribute].quantile(0.25)]['file_path'].tolist()

dir = r"tmp\music_vae_16bar\generated\iec0007\vector\gen_001\\"

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
# top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_files = [dir + path for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

# bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_files = [dir + path for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 属性ベクトルを求める
attribute_npy = np.mean(top_npy_arrays, axis=0) - np.mean(bottom_npy_arrays, axis=0)

print(attribute_npy)
np.save(attribute, attribute_npy)

In [ ]:
import pandas as pd
import numpy as np

filtered_csv_path = "tmp/filtered20240610.csv"
df = pd.read_csv(filtered_csv_path)

# c_diatonicの上位25%に属するfile_pathの一覧を配列で取得
top_25_percent = df[df['c_diatonic'] >= df['c_diatonic'].quantile(0.75)]['file_path'].tolist()

# c_diatonicの下位25%に属するfile_pathの一覧を配列で取得
bottom_25_percent = df[df['c_diatonic'] <= df['c_diatonic'].quantile(0.25)]['file_path'].tolist()

# file_path25個それぞれの末尾に.npyをつけて、np.loadする
top_npy_files = [path + '.npy' for path in top_25_percent]
top_npy_arrays = [np.load(npy_file) for npy_file in top_npy_files]

bottom_npy_files = [path + '.npy' for path in bottom_25_percent]
bottom_npy_arrays = [np.load(npy_file) for npy_file in bottom_npy_files]

# 上位25%平均を求める
average_npy = np.mean(top_npy_arrays, axis=0)

print(average_npy)


In [ ]:
import pandas as pd
import numpy as np

# CSVファイルの読み込み
df = pd.read_csv('tmp/result_20241106_212000.csv')

def calculate_arousal(row):
    # note_densityの正規化 (0-1の範囲に)
    density_normalized = (row['note_density'] - df['note_density'].min()) / \
                        (df['note_density'].max() - df['note_density'].min())
    
    # staccato_levelの正規化 (0-1の範囲に)
    staccato_normalized = (row['staccato_level'] - df['staccato_level'].min()) / \
                         (df['staccato_level'].max() - df['staccato_level'].min())
    
    # 両要素を組み合わせた総合Arousalスコア (各要素の重みは0.5ずつ)
    return (density_normalized * 0.5) + (staccato_normalized * 0.5)

# Arousalスコアを計算して新しい列として追加
df['arousal_score'] = df.apply(calculate_arousal, axis=1)